In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import pickle
from sklearn import svm
from tqdm import tqdm_notebook as tqdm
from random import shuffle
import itertools

In [2]:
cam = cv2.VideoCapture(0,cv2.CAP_DSHOW) 
cam_width =  cam.get(3)
cam_height = cam.get(4)
print(cam_width,cam_height)
# asl_img = cv2.imread('Media/M4/asl22.png')
# asl_img = cv2.resize(asl_img, (int(cam_width), int(cam_height)))
cam.release()

640.0 480.0


In [3]:
training_path='data'
cap = cv2.VideoCapture(0)
cv2.namedWindow("training")
classes=['1','2','3','4','5',"n"]
asci = [ord(item) for item in classes]
storage = None
override=False
while(True):

    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    k = cv2.waitKey(1)
    if k in asci:
        
        class_pos = asci.index(k)  
        
        class_name = classes[class_pos]
        image = frame[5:220,5:255] 
        image = cv2.resize(image, (int(cam_width), int(cam_height)))
        class_path= os.path.join("data",class_name)  
        if not os.path.exists(class_path):
            os.mkdir(class_path) 
        if storage != k:
            img_counter = 0
        if not override and img_counter == 0:
            if os.listdir(class_path):
                img_counter = max([int(img_name.replace(class_name,'').replace('.jpg','')) for img_name in os.listdir(class_path)])
        img_name = "{}{}.jpg".format(class_name, img_counter)

        # Save the image in folder named custom_training/<NAME_OF_CLASS>/  
        cv2.imwrite(os.path.join("data", '{}/{}'.format(class_name, img_name)), image)
        
        # Show the class name and the number of stored samples of the class
        cv2.putText(frame, "Class: {} , Sample Count: {}".format(class_name, img_counter), (10, 35),
                    cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 255, 0), 2)
        storage = k
        
        # Increment the counter for every new sample
        img_counter += 1
    cv2.rectangle(frame, (3, 3), (258, 223), (255,0,0), 2) 
    
    
    # Display the resulting frame
    cv2.imshow('training',frame )
    

    if cv2.waitKey(1)==27:
        break
    
cap.release()

cv2.destroyAllWindows()


In [4]:
train_classes = [directory for directory in os.listdir(training_path) if os.path.isdir(os.path.join(training_path,directory))]

In [5]:
# Define the percentage of data you want for the test set.
percentage = 10

for i in range(len(train_classes)):
    
    # Get the number of samples for each class
    length = len(os.listdir( os.path.join(training_path, train_classes[i] )))
    
    # Create a splitting point based on the percentage the user defined.
    splitter = round((length * percentage) / 100)
    
    # Print out the Training and Testing sample count of each class
    print('Total Samples in {}: {}, Train:{}, Test{}'.format(train_classes[i], length, (length-splitter), splitter))

# Create two dictionaries that will hold the training and test data
test_image_names  = {}
train_image_names = {}

# Now we will randomely shuffle our data and split between training and test data.
for i in range(len(train_classes)):
    
    # Get all the image names
    all_img_names = os.listdir(os.path.join(training_path, train_classes[i]))
    
    # Shuffle all the names
    shuffle(all_img_names)
    
    # Split the data
    train_image_names[train_classes[i]] =  all_img_names[splitter:]
    test_image_names[train_classes[i]]  =  all_img_names[:splitter]

Total Samples in 1: 51, Train:46, Test5
Total Samples in 2: 44, Train:40, Test4
Total Samples in 3: 60, Train:54, Test6
Total Samples in 4: 47, Train:42, Test5
Total Samples in 5: 56, Train:50, Test6
Total Samples in n: 108, Train:97, Test11


In [6]:
# Initialize hog parameters, I'm initializing with all the default values.
winSize = (120, 120) 
blockSize = (40, 40)
blockStride = (20, 20)
cellSize = (20, 20)
nbins = 9
derivAperture = 1 # has no effect 
winSigma = -1 # going to be equal to 4
histogramNormType = 1 # has no effect 
L2HysThreshold = 0.2
gammaCorrection = False
nlevels = 64 
signedGradient = False

hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins, derivAperture, winSigma, histogramNormType, 
                        L2HysThreshold, gammaCorrection, nlevels, signedGradient)

In [7]:
def getfeaturevectors(dictionary):
    # This list will hold the final feature vectors.
    Hog_feature_vectors = []

    # This list will hold the labels for each image.
    labels = []
    
    # This loop loops for each class (in total we have 29 classes for this dataset), we're also using tqdm for a progress bar.
    for classe_name in tqdm(list(dictionary.keys())):

            # This loop loops for each sample in each class (each class have approximately)
            for img_name in dictionary[classe_name]:

                # Read the image
                image = cv2.imread(os.path.join(training_path, classe_name, img_name))

                # Resize the image, if the image is corrupted then leave it 
                try:
                    image = cv2.resize(image, (120, 120))
                    
                except:
                    #print('Error Readimg image {}'.format(os.path.join(path,classes[i], j)))
                    continue

                # Compute the descriptor            
                descriptor = hog.compute(image)

                # Store the feature vector and the label
                Hog_feature_vectors.append(descriptor.reshape(-1))

                # Also store the label for each class
                labels.append(classe_name)

    print('Total Descriptors: {} and Total Labels: {}'.format(len(Hog_feature_vectors), len(labels) ))
    
    # Return the feature vectors with their labels.
    return Hog_feature_vectors, labels

In [8]:
hog_feature_vectors, labels = getfeaturevectors(train_image_names)

<ipython-input-7-8a181c4eafa5>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for classe_name in tqdm(list(dictionary.keys())):


  0%|          | 0/6 [00:00<?, ?it/s]

Total Descriptors: 300 and Total Labels: 300


In [9]:
%%time
model = svm.SVC(kernel='rbf', C = 30.0, gamma='scale')

model.fit(hog_feature_vectors, labels)
print("Training Completed")

Training Completed
Wall time: 51.3 ms


In [10]:
hog_feature_test_vectors, test_labels = getfeaturevectors(test_image_names)

<ipython-input-7-8a181c4eafa5>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for classe_name in tqdm(list(dictionary.keys())):


  0%|          | 0/6 [00:00<?, ?it/s]

Total Descriptors: 66 and Total Labels: 66


In [11]:
score = model.score(hog_feature_test_vectors, test_labels)
print('Accuracy of Model is {:.2f}%'.format(score*100))

Accuracy of Model is 100.00%


In [12]:
pickle.dump(model, open('data/svm_asl_custom.sav', 'wb'))

In [13]:
# load the model from disk
loaded_model = pickle.load(open('data/svm_asl_custom.sav', 'rb'))

In [ ]:
# Read from camera
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW);
a=""
while (True):
    ret ,frame = cap.read()
    if not ret:
        break
        
    frame = cv2.flip(frame, 1)
    cv2.rectangle(frame, (3, 3), (258, 223), (255,0,0), 2) 
#     cv2.rectangle(frame, (383, 4), (638, 223), (255,0,0), 2)
#     image = frame[1:230,380:620]
    image = frame[5:220,5:255]
    # Resize the image as required.
    image = cv2.resize(image, (120, 120))
    
    # Compute the descriptor            
    descriptor = hog.compute(image)
    
    # Extract the label
    label = loaded_model.predict(descriptor.reshape(1,-1))[0]    
    
    # Put the label and show the results
    cv2.putText(frame, 'Prediction: {}'.format(label), (10, 35), cv2.FONT_HERSHEY_SIMPLEX, 1.0,(100, 55, 200), 3)  
    #
    #
    #
   
    if label!=a:
        file= open('file.txt','a')
        if label=="n":
            pass
        else:
            file.write(label)
            print(label)
        a=label
    #
    #
    #
#     stacked = np.hstack((frame, full_stack))
    cv2.imshow("Results", frame)
    
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()